In [1]:
#Cargar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Suppress the FutureWarning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Opciones
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("autos_neoauto.csv")

In [3]:
print("Filas y columnas:", df.shape)

Filas y columnas: (3084, 7)


In [4]:
df.head(5)

,Marca,Modelo,Año,Precio,Combustible,Transmisión,Kilometraje
0,Mini,Hatch Cooper S,2022,27900.0,Gasolina,Automática - Secuencial,20550.0
1,Ford,Explorer,2022,55000.0,Gasolina,Automática - Secuencial,21332.0
2,Toyota,Yaris Cross,2024,21000.0,Gasolina-Hibrido,Automática - Secuencial,19841.0
3,Subaru,Impreza Sport,2024,24000.0,Gasolina,Automática,1000.0
4,Geely,Okavango,2023,19900.0,Hibrido,Automática,37462.0


In [5]:
df.tail(5)

,Marca,Modelo,Año,Precio,Combustible,Transmisión,Kilometraje
3079,Subaru,Impreza,2018,13400.0,Gasolina,Automática - Secuencial,42000.0
3080,Seat,Leon Style,2009,9000.0,Gasolina,Automática - Secuencial,134000.0
3081,Honda,Civic Lx,2020,19000.0,Gasolina,Automática,46000.0
3082,Chevrolet,Onix Joy,2023,10500.0,Gasolina,Mecánica,31800.0
3083,Peugeot,207 Compact,2010,5200.0,Gasolina,Mecánica,113000.0


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3084 entries, 0 to 3083
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Marca        3084 non-null   object 
 1   Modelo       3084 non-null   object 
 2   Año          3084 non-null   int64  
 3   Precio       3080 non-null   float64
 4   Combustible  3084 non-null   object 
 5   Transmisión  3084 non-null   object 
 6   Kilometraje  3008 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 168.8+ KB
None


In [7]:
print("\nValores nulos por columna:\n", df.isnull().sum())


Valores nulos por columna:
 Marca           0
Modelo          0
Año             0
Precio          4
Combustible     0
Transmisión     0
Kilometraje    76
dtype: int64


In [8]:
print(df.loc[df['Kilometraje'].isnull()])

        Marca     Modelo   Año   Precio Combustible              Transmisión  \
413   Hyundai     Tucson  2024  29490.0    Gasolina  Automática - Secuencial   
437   Hyundai     Tucson  2024  29990.0    Gasolina                 Mecánica   
444   Hyundai     I20 Hb  2024  17990.0    Gasolina               Automática   
502      Ford    Mustang  1966  24500.0    Gasolina                 Mecánica   
696      Ford     Escape  2024  37990.0     Hibrido               Automática   
...       ...        ...   ...      ...         ...                      ...   
2073  Hyundai  Grand I10  2026  12600.0        Dual                 Mecánica   
2130      Kia    Picanto  2019   8900.0    Gasolina                 Mecánica   
2164  Pontiac   Firebird  1974  19900.0    Gasolina               Automática   
2268     Fiat        124  1968   9900.0    Gasolina                 Mecánica   
2716     Fiat    Fiorino  2013   3700.0    Gasolina                 Mecánica   

      Kilometraje  
413           NaN  

In [9]:
df['Kilometraje'].fillna(0, inplace=True)

In [10]:
df.dropna(inplace=True)

In [11]:
model_counts = df['Modelo'].value_counts()

In [12]:
models_to_keep = model_counts[model_counts < 5].index

In [13]:
df_single_occurrence_models = df[df['Modelo'].isin(models_to_keep)]
print(df_single_occurrence_models)

          Marca           Modelo   Año   Precio       Combustible  \
0          Mini   Hatch Cooper S  2022  27900.0          Gasolina   
9         Lexus   Ux 250H Luxury  2019  24990.0  Gasolina-Hibrido   
10        Lexus  Rx 450H F Sport  2023  56990.0  Gasolina-Hibrido   
11        Lexus              Lbx  2025  35490.0           Hibrido   
12       Toyota             Agya  2024  12990.0          Gasolina   
...         ...              ...   ...      ...               ...   
3072    Peugeot              207  2013   8500.0          Gasolina   
3080       Seat       Leon Style  2009   9000.0          Gasolina   
3081      Honda         Civic Lx  2020  19000.0          Gasolina   
3082  Chevrolet         Onix Joy  2023  10500.0          Gasolina   
3083    Peugeot      207 Compact  2010   5200.0          Gasolina   

                  Transmisión  Kilometraje  
0     Automática - Secuencial      20550.0  
9                  Automática      53129.0  
10                 Automática      2

In [14]:
df['Modelo'].value_counts()

,count
Modelo,
Hilux,42
Forester,40
Tucson,38
X-Trail,37
Cx-5,36
...,...
All New Legacy,1
Mirage Hatchback,1
S22,1


In [15]:
df["Combustible"] = df["Combustible"].str.replace("Hibrido", "Electrico")
df["Combustible"] = df["Combustible"].str.replace("Gasolina-Electrico", "Gasolina")
df["Combustible"] = df["Combustible"].str.replace("ElÃ©ctrico", "Electrico")
df["Combustible"] = df["Combustible"].str.replace("Gas GLP", "Gas")
df["Combustible"] = df["Combustible"].str.replace("Gas GNV", "Gas")
df["Combustible"] = df["Combustible"].str.replace("Dual", "Gas")

In [16]:
df['Combustible'].value_counts()

,count
Combustible,
Gasolina,2516
Gas,287
Diesel,212
Electrico,65


In [17]:
df["Transmisión"] = df["Transmisión"].str.replace("Automática - Secuencial", "Automática")

In [18]:
df['Antiguedad'] = 2025 - df['Año']

In [19]:
df.drop(columns=['Año'], inplace=True)

In [20]:
df['Antiguedad'] = df['Antiguedad'].astype(int)
df['Precio'] = df['Precio'].astype(int)
df['Kilometraje'] = df['Kilometraje'].astype(int)

In [21]:
df

,Marca,Modelo,Precio,Combustible,Transmisión,Kilometraje,Antiguedad
0,Mini,Hatch Cooper S,27900,Gasolina,Automática,20550,3
1,Ford,Explorer,55000,Gasolina,Automática,21332,3
2,Toyota,Yaris Cross,21000,Gasolina,Automática,19841,1
3,Subaru,Impreza Sport,24000,Gasolina,Automática,1000,1
4,Geely,Okavango,19900,Electrico,Automática,37462,2
...,...,...,...,...,...,...,...
3079,Subaru,Impreza,13400,Gasolina,Automática,42000,7
3080,Seat,Leon Style,9000,Gasolina,Automática,134000,16
3081,Honda,Civic Lx,19000,Gasolina,Automática,46000,5
3082,Chevrolet,Onix Joy,10500,Gasolina,Mecánica,31800,2


In [22]:
numeric_df = df.select_dtypes(include=np.number)

In [23]:
correlation_matrix = numeric_df.corr()

In [24]:
correlation_matrix

,Precio,Kilometraje,Antiguedad
Precio,1.000000,-0.346468,-0.312004
Kilometraje,-0.346468,1.000000,0.545599
Antiguedad,-0.312004,0.545599,1.000000


In [25]:
df.drop(columns=['Modelo'], inplace=True)

In [26]:
df

,Marca,Precio,Combustible,Transmisión,Kilometraje,Antiguedad
0,Mini,27900,Gasolina,Automática,20550,3
1,Ford,55000,Gasolina,Automática,21332,3
2,Toyota,21000,Gasolina,Automática,19841,1
3,Subaru,24000,Gasolina,Automática,1000,1
4,Geely,19900,Electrico,Automática,37462,2
...,...,...,...,...,...,...
3079,Subaru,13400,Gasolina,Automática,42000,7
3080,Seat,9000,Gasolina,Automática,134000,16
3081,Honda,19000,Gasolina,Automática,46000,5
3082,Chevrolet,10500,Gasolina,Mecánica,31800,2


In [27]:
# Separar variables numéricas y categóricas
cat_features = ['Marca', 'Combustible', 'Transmisión']
num_features = ['Kilometraje', 'Antiguedad']

# Preprocesamiento
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ('num', StandardScaler(), num_features)
])


In [28]:
from sklearn.linear_model import LinearRegression, Ridge

In [29]:
models = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=0),
        'params': {
            'model__n_estimators': [100, 200],
            'model__max_depth': [None, 10]
        }
    },
    'LinearRegression': {
        'model': LinearRegression(),
        'params': {}  # No tiene hiperparámetros principales
    },
    'Ridge': {
        'model': Ridge(),
        'params': {
            'model__alpha': [0.1, 1.0, 10.0]
        }
    },
    'GradientBoosting': {
        'model': GradientBoostingRegressor(random_state=0),
        'params': {
            'model__n_estimators': [100, 200],
            'model__learning_rate': [0.1, 0.05]
        }
    }
}


In [30]:
X = df[['Marca', 'Combustible', 'Transmisión', 'Kilometraje', "Antiguedad"]]
y = df['Precio']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
resultados = {}

for name, m in models.items():
    pipe = Pipeline([
        ('pre', preprocessor),
        ('model', m['model'])
    ])

    grid = GridSearchCV(pipe, m['params'], cv=5, scoring='neg_mean_squared_error')
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_

    # Cross validation RMSE en el train set
    scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

    # Evaluación final en test
    y_pred = best_model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))

    resultados[name] = {
        'mejor_modelo': best_model,
        'RMSE_train_prom': np.sqrt(-scores.mean()),
        'RMSE_test': rmse_test,
        'mejores_params': grid.best_params_
    }

In [33]:
for nombre, res in resultados.items():
    print(f"\nModelo: {nombre}")
    print(f"  RMSE (train, CV promedio): {res['RMSE_train_prom']:.2f}")
    print(f"  RMSE (test): {res['RMSE_test']:.2f}")
    print(f"  Mejores parámetros: {res['mejores_params']}")


Modelo: RandomForest
  RMSE (train, CV promedio): 15543.00
  RMSE (test): 13949.85
  Mejores parámetros: {'model__max_depth': 10, 'model__n_estimators': 100}

Modelo: LinearRegression
  RMSE (train, CV promedio): 17101.47
  RMSE (test): 14675.85
  Mejores parámetros: {}

Modelo: Ridge
  RMSE (train, CV promedio): 17098.93
  RMSE (test): 14674.63
  Mejores parámetros: {'model__alpha': 0.1}

Modelo: GradientBoosting
  RMSE (train, CV promedio): 14885.21
  RMSE (test): 12955.37
  Mejores parámetros: {'model__learning_rate': 0.1, 'model__n_estimators': 200}


In [34]:
mejor_modelo = resultados['GradientBoosting']['mejor_modelo']

y_pred = mejor_modelo.predict(X_test)

for real, pred in zip(y_test[:10], y_pred[:10]):
    print(f"Real: {real:.2f} - Predicho: {pred:.2f}")

Real: 36900.00 - Predicho: 39725.69
Real: 66000.00 - Predicho: 66497.20
Real: 32999.00 - Predicho: 37718.34
Real: 31990.00 - Predicho: 35197.60
Real: 40900.00 - Predicho: 24808.36
Real: 16900.00 - Predicho: 16579.30
Real: 39990.00 - Predicho: 34285.50
Real: 14990.00 - Predicho: 24511.71
Real: 30900.00 - Predicho: 26394.85
Real: 20500.00 - Predicho: 18508.45


In [35]:
nuevos_autos = pd.DataFrame([
    {
        'Marca': 'Toyota',
        'Combustible': 'Gasolina',
        'Transmisión': 'Automática',
        'Kilometraje': 50000,
        'Antiguedad': 3
    },
    {
        'Marca': 'Hyundai',
        'Combustible': 'Gasolina',
        'Transmisión': 'Manual',
        'Kilometraje': 80000,
        'Antiguedad': 5
    }
])

# Predecir precios
precios_predichos = mejor_modelo.predict(nuevos_autos)

# Mostrar resultados
for i, precio in enumerate(precios_predichos):
    print(f"Auto {i+1}: Precio estimado = {precio:.2f}")

Auto 1: Precio estimado = 29414.22
Auto 2: Precio estimado = 17721.18


In [37]:
import joblib

# Guardar el mejor modelo entrenado
joblib.dump(resultados['GradientBoosting']['mejor_modelo'], 'modelo_autos.pkl')

['modelo_autos.pkl']